In [1]:
import asyncio
from dotenv import load_dotenv
from minference.threads.inference import InferenceOrchestrator, RequestLimits
from minference.threads.models import ChatMessage, ChatThread, LLMConfig, CallableTool, LLMClient,ResponseFormat, SystemPrompt, StructuredTool, Usage,GeneratedJsonObject
from typing import Literal, List
from minference.ecs.caregistry import CallableRegistry
import time
from minference.clients.utils import msg_dict_to_oai, msg_dict_to_anthropic, parse_json_string
from minference.ecs.entity import EntityRegistry
import os
import logging


In [20]:
all_monsters = json.load(open(r"/Users/tommasofurlanello/Documents/Dev/MarketInference/examples/all_monsters.json"))


In [22]:
all_monsters

{'monster': [{'name': 'Animated Coffin',
   'source': 'AATM',
   'size': ['L'],
   'type': 'construct',
   'alignment': ['U'],
   'ac': [{'ac': 16, 'from': ['natural armor']}],
   'hp': {'average': 60, 'formula': '8d10 + 16'},
   'speed': {'walk': 30, 'climb': 30},
   'str': 16,
   'dex': 12,
   'con': 15,
   'int': 3,
   'wis': 11,
   'cha': 3,
   'senses': ["blindsight 60 ft. (can't see beyond this radius)"],
   'passive': 10,
   'immune': ['poison', 'psychic'],
   'conditionImmune': ['blinded',
    'charmed',
    'deafened',
    'exhaustion',
    'frightened',
    'paralyzed',
    'petrified',
    'poisoned'],
   'languages': ["understands the languages of its creator but can't speak"],
   'cr': '3',
   'trait': [{'name': 'False Appearance',
     'entries': ["If the animated coffin is motionless at the start of combat, it has advantage on its initiative roll. Moreover, if a creature hasn't observed the coffin move or act, that creature must succeed on a {@dc 18} Intelligence ({@skil

In [2]:
load_dotenv()
EntityRegistry()
CallableRegistry()
oai_request_limits = RequestLimits(max_requests_per_minute=10000, max_tokens_per_minute=200000000)
lite_llm_request_limits = RequestLimits(max_requests_per_minute=500, max_tokens_per_minute=200000)
anthropic_request_limits = RequestLimits(max_requests_per_minute=50, max_tokens_per_minute=20000)


In [3]:
orchestrator = InferenceOrchestrator(oai_request_limits=oai_request_limits, litellm_request_limits=lite_llm_request_limits, anthropic_request_limits=anthropic_request_limits)
EntityRegistry.set_inference_orchestrator(orchestrator)
EntityRegistry.set_tracing_enabled(False)

In [4]:
import requests
import json
import os

def download_bestiary_data():
    # Define URLs for GitHub raw content
    INDEX_URL = "https://raw.githubusercontent.com/5etools-mirror-3/5etools-src/main/data/bestiary/index.json"
    BASE_URL = "https://raw.githubusercontent.com/5etools-mirror-3/5etools-src/main/data/bestiary/"
    
    # Create directory for downloaded files
    os.makedirs("bestiary_files", exist_ok=True)
    
    # Check if index file exists locally
    if os.path.exists("bestiary_files/index.json"):
        print("Loading bestiary index from local file...")
        with open("bestiary_files/index.json", "r", encoding="utf-8") as f:
            index_data = json.load(f)
        print(f"Index loaded successfully. Found {len(index_data)} entries.")
    else:
        print("Downloading bestiary index...")
        try:
            # Get the index file
            response = requests.get(INDEX_URL)
            response.raise_for_status()
            index_data = response.json()
            
            # Save the index file
            with open("bestiary_files/index.json", "w", encoding="utf-8") as f:
                json.dump(index_data, f, indent=2)
                
            print(f"Index downloaded successfully. Found {len(index_data)} entries.")
        except Exception as e:
            print(f"Error downloading index: {e}")
            return None
    
    # Filter for only bestiary files
    bestiary_files = {key: filename for key, filename in index_data.items() 
                     if filename.startswith("bestiary-")}
    
    print(f"Found {len(bestiary_files)} bestiary files to process.")
    
    # Download each bestiary file (if it doesn't exist locally)
    all_monsters = []
    failed_files = []
    
    for key, filename in bestiary_files.items():
        local_path = f"bestiary_files/{filename}"
        
        # Check if file already exists locally
        if os.path.exists(local_path):
            print(f"Loading {filename} from local file...")
            try:
                with open(local_path, "r", encoding="utf-8") as f:
                    file_data = json.load(f)
            except Exception as e:
                print(f"  Error loading local file {filename}: {e}")
                failed_files.append(filename)
                continue
        else:
            file_url = BASE_URL + filename
            print(f"Downloading {filename}...")
            
            try:
                response = requests.get(file_url)
                response.raise_for_status()
                file_data = response.json()
                
                # Save the file locally
                with open(local_path, "w", encoding="utf-8") as f:
                    json.dump(file_data, f, indent=2)
            except Exception as e:
                print(f"  Error downloading {filename}: {e}")
                failed_files.append(filename)
                continue
        
        # Extract monster data if present
        if "monster" in file_data:
            monsters_in_file = file_data["monster"]
            all_monsters.extend(monsters_in_file)
            print(f"  Added {len(monsters_in_file)} monsters from {filename}")
        else:
            print(f"  No monsters found in {filename}")
    
    # Create combined monster dictionary
    combined_data = {"monster": all_monsters}
    
    # Save combined data
    with open("all_monsters.json", "w", encoding="utf-8") as f:
        json.dump(combined_data, f, indent=2)
    
    print(f"\nProcessing complete!")
    print(f"Successfully processed {len(bestiary_files) - len(failed_files)} files")
    print(f"Combined {len(all_monsters)} monsters into 'all_monsters.json'")
    
    if failed_files:
        print(f"Failed to process {len(failed_files)} files: {', '.join(failed_files)}")
    
    return combined_data

In [5]:
combined_data = download_bestiary_data()

Loading bestiary index from local file...
Index loaded successfully. Found 103 entries.
Found 103 bestiary files to process.
Loading bestiary-aatm.json from local file...
  Added 3 monsters from bestiary-aatm.json
Loading bestiary-ai.json from local file...
  Added 22 monsters from bestiary-ai.json
Loading bestiary-aitfr-isf.json from local file...
  Added 2 monsters from bestiary-aitfr-isf.json
Loading bestiary-aitfr-thp.json from local file...
  Added 3 monsters from bestiary-aitfr-thp.json
Loading bestiary-aitfr-dn.json from local file...
  Added 4 monsters from bestiary-aitfr-dn.json
Loading bestiary-aitfr-fcd.json from local file...
  Added 2 monsters from bestiary-aitfr-fcd.json
Loading bestiary-awm.json from local file...
  Added 11 monsters from bestiary-awm.json
Loading bestiary-bam.json from local file...
  Added 72 monsters from bestiary-bam.json
Loading bestiary-bgdia.json from local file...
  Added 53 monsters from bestiary-bgdia.json
Loading bestiary-bgg.json from local f

In [6]:
len(combined_data['monster'])

4893

In [7]:
combined_data['monster'][15]["name"]

'Oak Truestrike'

In [8]:
named_dict = {}

for monster in combined_data['monster']:
    named_dict[monster['name']] = monster

In [9]:
named_dict.keys()

dict_keys(['Animated Coffin', 'Factol Skall', 'Heralds of Dust Exorcist', 'Ancient Deep Crow', 'Auspicia Dran', 'Brahma Lutier', 'Chaos Quadrapod', 'Clockwork Dragon', 'Deep Crow', "Donaar Blit'zen", 'Flabbergast', 'Jim Darkmagic', "K'thriss Drow'b", 'Keg Robot', 'Môrgæn', 'Oak Truestrike', 'Omin Dran', 'Pendragon Beestinger', 'Phoenix Anvil', 'Portentia Dran', 'Prophetess Dran', 'Rosie Beestinger', 'Splugoth the Returned', 'Viari', 'Walnut Dankgrass', 'Exul', 'Malivar', 'Greater Shadow Horror', 'Morwena Veilmist', 'Usagt', 'Giant Zombie Constrictor Snake', 'Hamish Hewland', 'Kyrilla, Accursed Gorgon', 'Skeletal Horror', 'Mercenary Envoy', 'Tyreus, Illusionist', 'Big Water Slurpent', 'Birdsquirrel', 'Boontu Monkey', 'Crystal Cave Merfolk', 'Dankwood Hag', 'Dum-Dum Goblin', 'Dwarf', 'Great Kroom, Purple Worm', 'Hangry Otyugh', 'Hill Giant, Blorbo', 'Saleeth the Couatl', 'Aartuk Elder', 'Aartuk Starhorror', 'Aartuk Weedling', 'Adult Lunar Dragon', 'Adult Solar Dragon', 'Ancient Lunar Dra

In [10]:
monster_schema={
  "type": "object",
  "title": "Monster",
  "description": "Essential D&D 5e monster stats with numerical combat values",
  "properties": {
    "name": {
      "type": "string",
      "description": "The monster's name"
    },
    "source": {
      "type": ["string", "null"],
      "description": "Source book or material"
    },
    "size": {
      "type": ["string", "null"],
      "enum": ["T", "S", "M", "L", "H", "G"],
      "description": "Size category (Tiny to Gargantuan)"
    },
    "type": {
      "type": ["string", "null"],
      "enum": [
        "aberration", "beast", "celestial", "construct", "dragon", 
        "elemental", "fey", "fiend", "giant", "humanoid",
        "monstrosity", "ooze", "plant", "undead"
      ],
      "description": "Creature type"
    },
    "armor_class": {
      "type": ["integer", "null"],
      "description": "Base armor class"
    },
    "hit_points": {
      "type": ["integer", "null"],
      "description": "Average hit points"
    },
    "hit_dice": {
      "type": "object",
      "description": "Hit dice details",
      "properties": {
        "count": {
          "type": ["integer", "null"],
          "description": "Number of dice (e.g., 3 in 3d8+6)"
        },
        "size": {
          "type": ["integer", "null"],
          "description": "Size of dice (e.g., 8 in 3d8+6)"
        },
        "modifier": {
          "type": ["integer", "null"],
          "description": "Modifier (e.g., 6 in 3d8+6)"
        }
      },
      "required": ["count", "size", "modifier"],
      "additionalProperties": False
    },
    "ability_scores": {
      "type": "object",
      "description": "The six ability scores",
      "properties": {
        "str": {
          "type": ["integer", "null"],
          "description": "Strength score"
        },
        "dex": {
          "type": ["integer", "null"],
          "description": "Dexterity score"
        },
        "con": {
          "type": ["integer", "null"],
          "description": "Constitution score"
        },
        "int": {
          "type": ["integer", "null"],
          "description": "Intelligence score"
        },
        "wis": {
          "type": ["integer", "null"],
          "description": "Wisdom score"
        },
        "cha": {
          "type": ["integer", "null"],
          "description": "Charisma score"
        }
      },
      "required": ["str", "dex", "con", "int", "wis", "cha"],
      "additionalProperties": False
    },
    "speeds": {
      "type": "object",
      "description": "Movement speeds in feet",
      "properties": {
        "walk": {
          "type": "integer",
          "description": "Walking speed"
        },
        "fly": {
          "type": ["integer", "null"],
          "description": "Flying speed"
        },
        "swim": {
          "type": ["integer", "null"],
          "description": "Swimming speed"
        },
        "climb": {
          "type": ["integer", "null"],
          "description": "Climbing speed"
        },
        "burrow": {
          "type": ["integer", "null"],
          "description": "Burrowing speed"
        }
      },
      "required": ["walk", "fly", "swim", "climb", "burrow"],
      "additionalProperties": False
    },
    "senses": {
      "type": "object",
      "description": "Special senses in feet",
      "properties": {
        "darkvision": {
          "type": ["integer", "null"],
          "description": "Darkvision range"
        },
        "blindsight": {
          "type": ["integer", "null"],
          "description": "Blindsight range"
        },
        "tremorsense": {
          "type": ["integer", "null"],
          "description": "Tremorsense range"
        },
        "truesight": {
          "type": ["integer", "null"],
          "description": "Truesight range"
        },
        "passive_perception": {
          "type": ["integer", "null"],
          "description": "Passive Perception score"
        }
      },
      "required": ["darkvision", "blindsight", "tremorsense", "truesight", "passive_perception"],
      "additionalProperties": False
    },
    "resistances": {
      "type": "array",
      "description": "Damage resistances",
      "items": {
        "type": "string",
        "enum": [
          "acid", "bludgeoning", "cold", "fire", "force", "lightning", 
          "necrotic", "piercing", "poison", "psychic", "radiant", 
          "slashing", "thunder"
        ]
      }
    },
    "immunities": {
      "type": "array",
      "description": "Damage immunities",
      "items": {
        "type": "string",
        "enum": [
          "acid", "bludgeoning", "cold", "fire", "force", "lightning", 
          "necrotic", "piercing", "poison", "psychic", "radiant", 
          "slashing", "thunder"
        ]
      }
    },
    "vulnerabilities": {
      "type": "array",
      "description": "Damage vulnerabilities",
      "items": {
        "type": "string",
        "enum": [
          "acid", "bludgeoning", "cold", "fire", "force", "lightning", 
          "necrotic", "piercing", "poison", "psychic", "radiant", 
          "slashing", "thunder"
        ]
      }
    },
    "condition_immunities": {
      "type": "array",
      "description": "Condition immunities",
      "items": {
        "type": "string",
        "enum": [
          "blinded", "charmed", "deafened", "exhaustion", "frightened",
          "grappled", "incapacitated", "invisible", "paralyzed", 
          "petrified", "poisoned", "prone", "restrained", "stunned", 
          "unconscious"
        ]
      }
    },
    "special_traits": {
      "type": "array",
      "description": "Special abilities and traits",
      "items": {
        "type": "object",
        "properties": {
          "name": {
            "type": "string",
            "description": "Name of the trait"
          },
          "description": {
            "type": "string",
            "description": "Description of the trait"
          }
        },
        "required": ["name", "description"],
        "additionalProperties": False
      }
    },
    "actions": {
      "type": "array",
      "description": "Actions the monster can take",
      "items": {
        "type": "object",
        "properties": {
          "name": {
            "type": "string",
            "description": "Name of the action"
          },
          "type": {
            "type": ["string", "null"],
            "enum": ["melee", "ranged", "both", "other"],
            "description": "Type of attack"
          },
          "attack_bonus": {
            "type": ["integer", "null"],
            "description": "Attack bonus to hit"
          },
          "reach": {
            "type": ["integer", "null"],
            "description": "Reach in feet for melee attacks"
          },
          "range": {
            "type": ["integer", "null"],
            "description": "Normal range for ranged attacks"
          },
          "damage_dice_count": {
            "type": ["integer", "null"],
            "description": "Number of dice (e.g., 2 in 2d6)"
          },
          "damage_dice_size": {
            "type": ["integer", "null"],
            "description": "Size of dice (e.g., 6 in 2d6)"
          },
          "damage_bonus": {
            "type": ["integer", "null"],
            "description": "Static damage bonus (e.g., 3 in 2d6+3)"
          },
          "damage_type": {
            "type": ["string", "null"],
            "enum": [
              "acid", "bludgeoning", "cold", "fire", "force", "lightning", 
              "necrotic", "piercing", "poison", "psychic", "radiant", 
              "slashing", "thunder"
            ],
            "description": "Type of damage dealt"
          },
          "save_dc": {
            "type": ["integer", "null"],
            "description": "Save DC if applicable"
          },
          "save_ability": {
            "type": ["string", "null"],
            "enum": ["str", "dex", "con", "int", "wis", "cha"],
            "description": "Ability for saving throw"
          }
        },
        "required": ["name", "type", "attack_bonus", "reach", "range", 
                    "damage_dice_count", "damage_dice_size", "damage_bonus", 
                    "damage_type", "save_dc", "save_ability"],
        "additionalProperties": False
      }
    },
    "legendary_actions": {
      "type": "array",
      "description": "Legendary actions",
      "items": {
        "type": "object",
        "properties": {
          "name": {
            "type": "string",
            "description": "Name of the legendary action"
          },
          "description": {
            "type": "string",
            "description": "Description of the legendary action"
          },
          "cost": {
            "type": "integer",
            "description": "Action cost (usually 1-3)"
          }
        },
        "required": ["name", "description", "cost"],
        "additionalProperties": False
      }
    },
    "challenge_rating": {
      "type": ["number", "string", "null"],
      "description": "Challenge rating"
    },
    "xp": {
      "type": ["integer", "null"],
      "description": "XP value"
    },
    "languages": {
      "type": "array",
      "description": "Languages known",
      "items": {
        "type": "string"
      }
    }
  },
  "required": [
    "name", "source", "size", "type", "armor_class", "hit_points", 
    "hit_dice", "ability_scores", "speeds", "senses", "resistances", 
    "immunities", "vulnerabilities", "condition_immunities", "special_traits", 
    "actions", "legendary_actions", "challenge_rating", "xp", "languages"
  ],
  "additionalProperties": False
}

In [11]:
monster_extractor = StructuredTool(name="monster_extractor",json_schema=monster_schema)

In [12]:
print(monster_extractor.schema_instruction)

Please follow this JSON schema for your response:: {'type': 'object', 'title': 'Monster', 'description': 'Essential D&D 5e monster stats with numerical combat values', 'properties': {'name': {'type': 'string', 'description': "The monster's name"}, 'source': {'type': ['string', 'null'], 'description': 'Source book or material'}, 'size': {'type': ['string', 'null'], 'enum': ['T', 'S', 'M', 'L', 'H', 'G'], 'description': 'Size category (Tiny to Gargantuan)'}, 'type': {'type': ['string', 'null'], 'enum': ['aberration', 'beast', 'celestial', 'construct', 'dragon', 'elemental', 'fey', 'fiend', 'giant', 'humanoid', 'monstrosity', 'ooze', 'plant', 'undead'], 'description': 'Creature type'}, 'armor_class': {'type': ['integer', 'null'], 'description': 'Base armor class'}, 'hit_points': {'type': ['integer', 'null'], 'description': 'Average hit points'}, 'hit_dice': {'type': 'object', 'description': 'Hit dice details', 'properties': {'count': {'type': ['integer', 'null'], 'description': 'Number of

In [13]:
system_prompt = SystemPrompt(content="You are a epidemiologist expert that is studying monsters in an alternative universe that can extract information from a text and convert it into a structured output. You will be given a text and you will need to extract the information and convert it into a structured output. You will be given a text and you will need to extract the information and convert it into a structured output.", name= "monster_extractor")

In [14]:
llm_config=LLMConfig(client=LLMClient.openai, model="gpt-4o-mini", response_format=ResponseFormat.structured_output,max_tokens=4000)

thread = ChatThread(
    system_prompt=system_prompt,
    new_message="",
    llm_config=llm_config,
    forced_output=monster_extractor,
    use_schema_instruction=True
)
thread_id = thread.id
            

In [15]:
message = f"Give me the information about the monster {named_dict["Acolyte"]}"
thread.fork(force=True,**{"new_message":message})

ChatThread(id=UUID('cb6073be-19bf-45b8-9846-b0ab97a61f23'), live_id=UUID('fdf5e4c4-afc5-4a8f-b792-70a2723ddf2e'), created_at=datetime.datetime(2025, 3, 2, 14, 19, 16, 515923), parent_id=UUID('b6323532-5e3d-4cb2-96b2-3aa7f6cc64ba'), lineage_id=UUID('20bb9e10-ab01-475b-bc9a-a6124c934a10'), old_ids=[UUID('b6323532-5e3d-4cb2-96b2-3aa7f6cc64ba')], name=None, system_prompt=SystemPrompt(id=UUID('fcd8e640-65d1-4c48-9bc5-047122f003a1'), live_id=UUID('f68ff64c-b759-4229-8d7e-e8773b3c0ea5'), created_at=datetime.datetime(2025, 3, 2, 14, 19, 16, 511876), parent_id=None, lineage_id=UUID('9bb991d0-8b29-4f32-9f7e-4744f2ad88bf'), old_ids=[], name='monster_extractor', content='You are a epidemiologist expert that is studying monsters in an alternative universe that can extract information from a text and convert it into a structured output. You will be given a text and you will need to extract the information and convert it into a structured output. You will be given a text and you will need to extract 

In [16]:
thread_list = []
thread_dict = {}
for monster_name, monster_data in named_dict.items():
    base_thread  = EntityRegistry.get(thread_id)
    assert base_thread is not None
    base_thread.fork(force=True,**{"new_message":f"Please study in detail the monster {monster_data}"})
    thread_list.append(base_thread.id)
    thread_dict[monster_name] = base_thread.id

In [17]:
threads = EntityRegistry.get_many(thread_list)

In [18]:
test_threads = threads[:1000]

In [19]:
test_outs = await orchestrator.run_parallel_ai_completion(test_threads)

INFO:root:Starting request #0
INFO:root:Starting request #1
INFO:root:Starting request #2
INFO:root:Starting request #3
INFO:root:Starting request #4
INFO:root:Starting request #5
INFO:root:Starting request #6
INFO:root:Starting request #7
INFO:root:Starting request #8
INFO:root:Starting request #9
INFO:root:Starting request #10
INFO:root:Starting request #11
INFO:root:Starting request #12
INFO:root:Starting request #13
INFO:root:Starting request #14
INFO:root:Starting request #15
INFO:root:Starting request #16
INFO:root:Starting request #17
INFO:root:Starting request #18
INFO:root:Starting request #19
INFO:root:Starting request #20
INFO:root:Starting request #21
INFO:root:Starting request #22
INFO:root:Starting request #23
INFO:root:Starting request #24
INFO:root:Starting request #25
INFO:root:Starting request #26
INFO:root:Starting request #27
INFO:root:Starting request #28
INFO:root:Starting request #29
INFO:root:Starting request #30
INFO:root:Starting request #31
INFO:root:Starting

CancelledError: 

In [20]:
generations = EntityRegistry.list_by_type(GeneratedJsonObject)
len(generations)

100

In [21]:
generations[777].object

IndexError: list index out of range